# The WGBS data analysis with new data
## Overview 
In this notebook, we are going to explore how to run this module with a new dataset. These submodules provide a great framework for running a rigorous and scalable methylseq analysis, but there are some considerations that must be made in order to run this with your own data. We will walk through that process here so that hopefully, you are able to take these notebooks to your research group and use them for your own analysis. Notice that we do not give you all the answers in the code blocks, but if you get stuck, use the dropdowns for help. We will walk through it together in Tutorial 1 to run the Bismark stage of analysis. After you complete this tutorial, try to use the same logic to build out the other tutorials with this new dataset.

## Learning Objectives

* **Setting up a bioinformatics analysis environment:** Learners will practice installing necessary bioinformatics tools (FastQC, MultiQC, Bismark, Trim Galore!, Bedtools, Samtools, Metilene) using conda/mamba within a Jupyter Notebook environment.  This includes managing software versions and dependencies.

* **Data Acquisition and Management:**  Learners will download a WGBS (Whole Genome Bisulfite Sequencing) dataset and reference genome from a Google Cloud Storage bucket using `gsutil` commands, organizing files into a project directory structure. This reinforces best practices for handling large datasets and reproducible research.

* **WGBS Data Preprocessing:** Learners will use FastQC to assess the quality of raw sequencing data, then use Trim Galore! for adapter trimming and quality filtering of the sequencing reads. They'll learn to interpret quality control reports generated by FastQC and summarize the reports with MultiQC.


* **Bismark Pipeline Execution:** Learners will execute the Bismark pipeline for alignment of bisulfite-converted reads to a reference genome. This includes preparing the reference genome using Bismark's genome preparation tools, aligning reads using Bismark's aligner and generating reports for analysis.


* **Bismark Data Processing and Quality Control:** They will perform deduplication of Bismark alignments using `deduplicate_bismark`. They'll then extract methylation ratios from Bismark alignment files to generate bedGraph and coverage files using `bismark_methylation_extractor`. Learners also perform quality control checks on generated files.


* **Data Visualization with IGV:**  Learners will utilize the `igv-notebook` library to visualize the aligned BAM files and methylation levels (bedGraph files) in the Integrative Genomics Viewer (IGV).  This includes loading the appropriate genome, alignment files and metadata within the IGV interface.

## Prerequisites

**1. Software:**

* **Mamba (or conda):** A package manager for installing and managing software dependencies.  The notebook uses it to install the bioinformatics tools.
* **Bioconda channel:** A conda channel containing bioinformatics tools.  Must be specified when installing the tools.
* **FastQC:**  Quality control for high throughput sequence data.
* **MultiQC:** Aggregates results from multiple FastQC reports into a single report.
* **Bismark:** Aligns bisulfite-treated sequencing reads to a reference genome.
* **Trim Galore!:**  Trims adapters and low-quality sequences from reads.
* **Bedtools:** A suite of utilities for genomic analysis.
* **Samtools:**  Utilities for manipulating SAM/BAM files (sequence alignment/mapping format).
* **Metilene:**  Software for methylation analysis (although its use is not explicitly shown in this notebook excerpt).

**2. APIs:**

* **Google Cloud Storage API:** Required for downloading the example dataset and reference genome from the specified Google Cloud Storage bucket using `gsutil`.

**Important Note:** The notebook contains placeholders like `<YOUR COMMAND HERE>`.  These need to be replaced with the appropriate commands based on the file paths and specific parameters required.  The help dropdowns provide examples; ensure those commands accurately match the setup and local system.  Remember to adapt file paths within commands to your local directory structure.

## Get Started

Use mamba to install the tools needed for this analysis: fastqc, multiqc bismark, trim-galore, bedtools, samtools, and metilene. Remember to set the channel to bioconda in order to access the bioinformatics tools.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`mamba install -y -c conda-forge -c bioconda fastqc multiqc bismark trim-galore bedtools samtools metilene` 


</details>

---
### **Importing the example dataset**

#### Set up the environment
As before, we are going to set up the directory structure to keep everything organized. We will change `Tutorial_1` to `Tutorial_5`. Use the `mdkir` commands to create your directory structure below.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`# Show current working directory`  
`pwd`  
`# Create directories`  
`mkdir -p Tutorial_5`  
`mkdir -p Tutorial_5/ref_genome`  
`mkdir -p Tutorial_5/fastqc`  
`mkdir -p Tutorial_5/trimmed`   
`mkdir -p Tutorial_5/fastq`  


</details>



#### About the new dataset

Our new dataset comes from a paper by [Hadad et al. Epigenetics Chromatin. 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6781367/) that compares methylation changes in mice as they age and correlates those changes to gene expression changes. The data is available in SRA under the bioProject number [PRJNA523985](https://www.ncbi.nlm.nih.gov/sra/?term=PRJNA523985). The impact of methylation on aging, particularly in brain tissue, is of great research interest. The paper makes several comparisons, but to keep this tutorial simple we only pulled the samples of young and old female mice. We also subsamples the data to 10% of the reads on SRA, otherwise the data would be very large and would require long compute times and higher costs than would be appropriate for a tutorial.


#### Download example dataset and reference genome from Google Cloud Storage bucket

This WGBS example dataset was stored in a **Google Cloud Storage bucket**. You can use `gsutil` commands to access and manage the cloud storage. 

To download this example dataset, we need to get the fastq.gz files, the samplesheet (.tsv), and the CpG island annotation file (.bed) from the storage bucket to the Tutorial_5 directory we just created using `gsutil cp`. In the cell below, type the commands to do this.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`# Download the example dataset from a Google Cloud Storage bucket`  
`gsutil cp gs://nigms-sandbox/dna-methyl/*.fastq.gz Tutorial_5/fastq`  
`# Download the samplesheet`  
`gsutil cp gs://nigms-sandbox/dna-methyl/*.tsv Tutorial_5/`  
`# Download the bed file`  
`gsutil cp gs://nigms-sandbox/dna-methyl/*.bed Tutorial_5/`  



</details>



Next download the **reference genome** in the same way and place it in the `ref_genome` folder. The genome should be in FASTA format, and can be downloaded from NCBI or Ensemble websites. As before, we will only use the sequences from GRCm39 chromosome 6.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`gsutil cp gs://nigms-sandbox/dna-methyl/Mus_musculus.GRCm39.dna.chromosome.6.fa Tutorial_5/ref_genome`


</details>



---
## **Running Bismark pipeline**

#### Step 1. FastQC 

As before, we start with [**FastQC**](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) to evaluate the quality of the sequence data we got from SRA. Write a command below to run FastQC on the new data.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`for file in Tutorial_5/fastq/*.gz; do fastqc -q -o Tutorial_5/fastqc "${file}"; done;`


</details>



#### Step 2. MultiQC (optional)
Next, we will use __[MultiQC](https://multiqc.info/)__ to aggregate results from across the samples into a single report. As before, save the output in `Tutorial_5/multiqc`. Below, write the commands to create the report and then view it as an html file.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`# Run multiqc to summarize all the fastqc reports`  
`multiqc -f -p  Tutorial_5/fastqc -o Tutorial_5/multiqc`  
  
`# View multiqc report`  
`from IPython.display import IFrame`  
`IFrame(src='Tutorial_5/multiqc/multiqc_report.html', width=1200, height=400)`  


</details>



#### Step 3. Trim the adapters and low quality sequence reads

Adapter sequences should be removed from reads because they interfere with downstream analyses, such as alignment of reads to a reference.

As before, we will use __[Trim Galore!](https://www.bioinformatics.babraham.ac.uk/projects/trim_galore/)__ for adaptor trimming and quality control. Using the Tutorial 1 trim_galore command as a template, write a command to run trim_galore on your reads. Use wildcards to match your fastq files.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`trim_galore -j 4 --paired --illumina -o Tutorial_5/trimmed --fastqc Tutorial_5/fastq/SRR*.gz`


</details>



#### Step 4. Genome preparation

In order to align the bisulfite-treated reads to reference genome, the reference genome need to be converted as well. 

Write the command below to prepare the reference genome. Do we need to change anything? Or can we leave it the same as with Tutorial 1? What factors do you need to consider to determine this?

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>
    Since we are using the same reference genome, the command stays the same. The only change we need to make is to the path where it is stored.

    

`bismark_genome_preparation Tutorial_5/ref_genome`

</details>



#### Step 5. Run Bismark (alignment)

We next run Bismark to align the reads to the reference genome. If you recall from Tutorial 1, the usage for Bismark is:

`bismark [options] --genome <genome_folder> {-1 <mates1> -2 <mates2> | <singles>}`

In the command block below, write your adapted Bismark command for your new dataset.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`for file_name in Tutorial_5/trimmed/*_R1_val_1.fq.gz; do \
    bismark --genome Tutorial_5/ref_genome/ -1 ${file_name} -2 ${file_name//_R1_val_1/_R2_val_2} -o Tutorial_5/bismark; \
done;`


</details>




#### Step 6. De-duplicate 

Recall from Tutorial 1 that a high level of duplication is more likely to indicate some kind of enrichment bias (eg PCR over amplification). To mitigate that potential bias, we will run a de-duplication process on our alignments.
 
> <img src="images/1_complex_library.png" width="500"/>   

In the command block below, write your `deduplicate_bismark` command with your new data.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`deduplicate_bismark -p --output_dir Tutorial_1/bismark --bam Tutorial_1/bismark/*.bam `


</details>

#### Step 7. Get methylation ratios 

This step operates on Bismark alignment files and extracts the methylation call for every single C analyzed. Think about what command is used to do this. What input files do you need? Do you need to change any flags for this run? What directory do you need to write the output to? Write your command in the cell below try it out.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`bismark_methylation_extractor --gzip --bedGraph --ignore_r2 2 -o Tutorial_1/bismark Tutorial_1/bismark/*deduplicated.bam`


</details>



As a sanity check, look at the first few lines of a couple output files.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`zcat Tutorial_5/bismark/SRX202087_R1_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz | head -n 10`  
`zcat Tutorial_5/bismark/SRX202087_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz | head -n 10`


</details>

#### Step 8. Generate report and summary 
Finally, generate a report and summary of the analysis.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>

```
cd Tutorial_5/bismark && bismark2report
cd Tutorial_5/bismark && bismark2summary  
# Display the final summary from bismark workflow, including all 4 samples
import pandas as pd
df = pd.read_csv("Tutorial_5/bismark/bismark_summary_report.txt", sep='\t')
df
```

</details>

#### Important output files for downstream analysis

The Bismark methylation extractor (step 7) can output files in `.bedGraph` and `.cov` format with the methylation levels (percentage of methylated cytosine) at each position. These files can be used as input for downstream analysis, such as identification of differentially methylated regions.


---
## **Visualization using IGV (Integrative Genomics Viewer)**

Let's look at our recently analyzed data with Integrative Genomics Viewer. Remember the three basic steps of using IGV as you load your new data.

#### Basic usage
- Select reference genome - IGV hosts dozens of genomes and you can load other genomes too
- Load data tracks
- Navigate

#### Install `igv-notebook` 

Install igv-notebook with pip.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`pip install igv-notebook`


</details>



Create a browser and load your reference genome.

In [ ]:
<YOUR COMMAND BELOW>

<details>
  <summary>Click for help</summary>


`import igv_notebook`  
`igv_notebook.init()`  

`b = igv_notebook.Browser(`  
`    {`  
`        "genome": "mm39",`  
`        "locus": "chr6:4,733,000-4,749,000"`  
`    }`  
`)`  


</details>



Next, load the data tracks. This requires a sorted and indexed bam files. Write the command below to do that.

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`for i in Tutorial_5/bismark/*deduplicated.bam; do samtools sort $i -o $i.sort.bam; samtools index $i.sort.bam;done`  
`for f in Tutorial_5/bismark/*sort.bam* ; do mv "$f" "$(echo "$f" | sed s/_R1_val_1_bismark_bt2_pe.deduplicated.bam//)"; done`  


</details>



#### Load and view the alignment file (and its index) to igv-notebook.

Start a new browser "b2", and load one of sample alignment .bam and its index file .bai into the browser:

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`b2 = igv_notebook.Browser(`  
`    {`  
`       "genome": "mm39",`  
`       "locus": "chr6:4,733,000-4,749,000"`  
`    }`  
`)`  
`b2.load_track({`  
`    "name": "SRX271141",` 
`    "path": "Tutorial_1/bismark/NAME.sort.bam",`  
`    "indexPath": "Tutorial_1/bismark/NAME.sort.bam.bai",` 
`    "format": "bam",`  
`    "type": "alignment",`  
`})`  

</details>




<div class=>

#### View the methylation levels (.bedgraph files)

Next, load the bedgraph files to view the methylation percentage at each position. 

In [ ]:
<YOUR COMMAND HERE>

<details>
  <summary>Click for help</summary>


`b3 = igv_notebook.Browser({"genome": "mm39","locus": "chr6:4,500,000-5,500,000"})`   
  
`b3.load_track({ "name": "SRX271141_bed","path": "Tutorial_1/bismark/SRX271141_R1_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz","mode":"bisulfite" ,"color": "orange"})`  
      
`b3.load_track({` 
`    "name": "SRX202087_bed",` 
`    "path": "Tutorial_1/bismark/SRX202087_R1_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz",` 
`    "mode": "bisulfite" ,` 
`    "color": "orange"` 
`})` 
  
`b3.load_track({` 
`    "name": "SRX271142_bed",` 
`    "path": "Tutorial_1/bismark/SRX271142_R1_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz",` 
`    "mode": "bisulfite" ,` 
`    "color": "blue"` 
`})` 
    
    
`b3.load_track({` 
`    "name": "SRX202088_bed",` 
`    "path": "Tutorial_1/bismark/SRX202088_R1_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz",` 
`    "mode": "bisulfite" ,` 
`    "color": "blue"` 
`})` 


</details>




Finally, zoom into a specific region to have a closer look.

<details>
  <summary>Click for help</summary>


`# Zoom in to see a specific region in the above browser`  
`type your region in the format chr<number>:start-end`  
`b3.search('<region_here>')`


</details>



## Conclusion

This Jupyter Notebook demonstrated the application of a WGBS data analysis module to a new dataset from Hadad et al. (2019), detailing the process of adapting the pipeline for novel data. We successfully reproduced the Bismark pipeline steps, including quality control with FastQC and MultiQC, adapter trimming with Trim Galore!, genome preparation, Bismark alignment, deduplication, methylation extraction, and report generation. The final methylation data was visualized using IGV-notebook, showcasing the workflow's effectiveness in analyzing WGBS data and providing a template for future analyses. While the tutorial focused on a specific dataset, the adaptability of the module is highlighted by its successful implementation with the new data, suggesting its potential for broad use in WGBS research.

## Clean Up
**Don't forget:** After you finish running the notebook, stop it in Vertex AI Workbench  to avoid accumulating costs.